In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
import torch.optim as optim

import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import data as dt, neuronshap as ns, sim
from cfgs.fedargs import *

from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
)
from libs.helpers.finance import bin_hours_per_week, bin_NATIVITY_level, test_RACIP_enum
from libs.helpers.metrics import (
    conditional_demographic_parity_difference,
    conditional_demographic_parity_ratio,
)
from libs.helpers.plot import group_box_plots

/Users/hkasyap/anaconda3/envs/menv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from folktables import ACSDataSource, ACSEmployment

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)
features, label, group = ACSEmployment.df_to_numpy(acs_data)

df = pd.DataFrame(features)
df.columns = ACSEmployment.features
df[ACSEmployment.target] = label

categorical_features = ['SCHL','MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC', 'DEAR', 'DEYE', 'DREM']
df = pd.get_dummies(df, columns = categorical_features)

numeric_features = ['AGEP']
ss = StandardScaler()
df[numeric_features] = ss.fit_transform(df[numeric_features])

#df[df['SEX'] == 1.0] = 1
df[df['SEX'] == 2.0] = 0
#df[df['RAC1P'] != 1.0] = 0
df['ESR'] = df['ESR'].astype('int')
df['SEX'] = df['SEX'].astype('int')
#df['RAC1P'] = df['RAC1P'].astype('int')
df['NATIVITY'] = df['NATIVITY'].astype('int')

df.head(10)

,AGEP,NATIVITY,SEX,RAC1P,ESR,SCHL_0.0,SCHL_1.0,SCHL_2.0,SCHL_3.0,SCHL_4.0,...,ANC_2.0,ANC_3.0,ANC_4.0,DEAR_1.0,DEAR_2.0,DEYE_1.0,DEYE_2.0,DREM_0.0,DREM_1.0,DREM_2.0
0,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.429499,1,1,1.0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
3,-0.620885,1,1,1.0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
4,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,-0.200731,1,1,1.0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,1
8,-0.074685,1,1,1.0,1,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,1
9,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
import copy

#https://nannyml.readthedocs.io/en/v0.9.0/datasets/ma_employment.html
train, test = train_test_split(df, test_size=0.2, random_state=42)


train_oh, test_oh = copy.deepcopy(train), copy.deepcopy(test)

test = test.reset_index(drop=True)
test_oh = test_oh.reset_index(drop=True)

#categorical_features = ['RAC1P']
#train_oh = pd.get_dummies(train_oh, columns = categorical_features)
#test_oh = pd.get_dummies(test_oh, columns = categorical_features)

train_oh.head()
#test.info()
#test_oh["NATIVITY"].value_counts()

,AGEP,NATIVITY,SEX,RAC1P,ESR,SCHL_0.0,SCHL_1.0,SCHL_2.0,SCHL_3.0,SCHL_4.0,...,ANC_2.0,ANC_3.0,ANC_4.0,DEAR_1.0,DEAR_2.0,DEYE_1.0,DEYE_2.0,DREM_0.0,DREM_1.0,DREM_2.0
25249,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26314,-0.914992,1,1,1.0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
22376,0.933683,1,1,1.0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
22654,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14182,0.000000,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
m_dh_oh = test_oh.loc[test_oh["SEX"] == 1]
m_dh_oh = m_dh_oh.head(100)
fm_dh_oh = test_oh.loc[test_oh["SEX"] == 0]
fm_dh_oh = fm_dh_oh.head(100)

In [5]:
X_train = train_oh.drop(columns="ESR").values
Y_train = train_oh['ESR'].values
X_test = test_oh.drop(columns="ESR").values
Y_test = test_oh['ESR'].values
X_m = m_dh_oh.drop(columns="ESR").values
Y_m = m_dh_oh['ESR'].values
X_fm = fm_dh_oh.drop(columns="ESR").values
Y_fm = fm_dh_oh['ESR'].values

#creating torch dataset and loader using original dataset. 
#to use resampled dataset, replace ex. xtrain with xtrain_over etc.
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train).float(), torch.tensor(Y_train).long())
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test).float(), torch.tensor(Y_test).long())
m_data = torch.utils.data.TensorDataset(torch.tensor(X_m).float(), torch.tensor(Y_m).long())
fm_data = torch.utils.data.TensorDataset(torch.tensor(X_fm).float(), torch.tensor(Y_fm).long())

train_loader = torch.utils.data.DataLoader(train_data,batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))
m_loader = torch.utils.data.DataLoader(m_data, batch_size=1)
fm_loader = torch.utils.data.DataLoader(fm_data, batch_size=1)

In [6]:
class BasicNet(torch.nn.Module):
    
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.layers = 0
        
        self.lin1 = torch.nn.Linear(self.num_features,  150)        
        self.lin2 = torch.nn.Linear(50, 50)        
        self.lin3 = torch.nn.Linear(50, 50)
        
        self.lin4 = torch.nn.Linear(150, 150) 
        
        self.lin5 = torch.nn.Linear(50, 50)        
        self.lin6 = torch.nn.Linear(50, 50)
        self.lin10 = torch.nn.Linear(150, self.num_classes)
        
        self.prelu = torch.nn.PReLU()
        self.dropout = torch.nn.Dropout(0.25)

    def forward(self, xin):
        self.layers = 0
        
        x = F.relu(self.lin1(xin))
        self.layers += 1
        
        #x = F.relu(self.lin2(x))
        #self.layers += 1
        for y in range(8):
            x = F.relu(self.lin4(x)) 
            self.layers += 1
           
        x = self.dropout(x)
        
        x = F.relu(self.lin10(x)) 
        self.layers += 1
        return x

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for inputs, target in train_loader:
      
        #inputs, target = inputs.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target.long())
        # Backprop
        loss.backward()
        optimizer.step()
        ###

In [8]:
def test(model, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    
    with torch.no_grad():
      
        for inputs, target in test_loader:
            
            #inputs, target = inputs.to(device), target.to(device)
            
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target.long()).item() 
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [9]:
model = BasicNet(88, 2)
test_accuracy = []
train_loss = []
nbr_epochs = 5
lr = 0.0025# 
weight_decay = 0

# Surrogate loss used for training
loss_fn = torch.nn.CrossEntropyLoss()
test_loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=lr ,weight_decay=weight_decay)
#optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

print('Training beginning...')
#start_time = time.time()

for epoch in range(1, nbr_epochs+1):
    print('Epoch ', epoch, ':')
    train(model, train_loader, optimizer, epoch)
    loss, acc = test(model, test_loader)
    
    # save results every epoch
    test_accuracy.append(acc)
    train_loss.append(loss)
    
#end_time = time.time()
#print('Training on ' + str(nbr_epochs) + ' epochs done in ', str(end_time-start_time),' seconds')

Training beginning...
Epoch  1 :

Test set: Average loss: 0.1725, Accuracy: 8783/9556 (92%)

Epoch  2 :

Test set: Average loss: 0.1687, Accuracy: 8833/9556 (92%)

Epoch  3 :

Test set: Average loss: 0.1829, Accuracy: 8806/9556 (92%)

Epoch  4 :

Test set: Average loss: 0.1702, Accuracy: 8830/9556 (92%)

Epoch  5 :

Test set: Average loss: 0.1692, Accuracy: 8827/9556 (92%)



In [10]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = model(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]

print(sum(Y_test), sum(Y_pred), sum(pred))


Accuracy: 8827/9556 (92%)

2032 tensor([2053]) tensor([2053])


<h1>Demographic Parity</h1>

<h2>Distribution of scores by sex</h2>

In [26]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test_oh.SEX,
)

dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.SEX,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.176
Demographic parity ratio: 0.000


In [27]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test_oh.NATIVITY,
)

dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.NATIVITY,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.221
Demographic parity ratio: 0.000


In [28]:
test_RAC1P_enum = test_oh.RAC1P.map(test_RACIP_enum)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test_oh.SEX, test_RAC1P_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test_oh.SEX, test_RAC1P_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.110
Conditional demographic parity ratio: 0.500


In [29]:
test_RAC1P_enum = test_oh.RAC1P.map(test_RACIP_enum)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test_oh.NATIVITY, test_RAC1P_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test_oh.NATIVITY, test_RAC1P_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.219
Conditional demographic parity ratio: 0.228


<h1>Equalised Odds</h1>

<h2>Distribution of scores by sex for high and low earners</h2>

In [30]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test_oh.SEX,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.SEX,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.360
Equalised odds ratio: 0.000


<h2>Distribution of scores by race for high and low earners</h2>

In [31]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test_oh.NATIVITY,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.NATIVITY,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.365
Equalised odds ratio: 0.000


<h1>Shapley based Neuron Pruning for Fairness</h1>

In [17]:
m_shapley_values = ns.calculate_shapley_values_fa(model, m_loader, 200)
print(m_shapley_values)
fm_shapley_values = ns.calculate_shapley_values_fa(model, fm_loader, 200)
print(fm_shapley_values)

[ 136.17596   105.508606    9.646261 ... 1739.7612   1522.7279
    0.      ]
[  0.       0.       0.     ... 597.6179   0.       0.    ]


In [32]:
diff_shap_values = m_shapley_values - fm_shapley_values
max_diff_shap_values_ind = np.argpartition(diff_shap_values, -150)[-150:]
#diff_shap_values[max_diff_shap_values_ind]

In [33]:
model_arr, model_slist = sim.get_net_arr(model)
model_arr[max_diff_shap_values_ind] = 0
updated_model = sim.get_arr_net(model, model_arr, model_slist)

In [34]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = updated_model(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))
        

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]
print(sum(Y_test), sum(Y_pred), sum(pred))


Accuracy: 7524/9556 (79%)

2032 tensor([0]) tensor([0])


In [21]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test_oh.NATIVITY,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.NATIVITY,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.221
Demographic parity ratio: 0.000


In [22]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test_oh.SEX,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test_oh.SEX,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.360
Equalised odds ratio: 0.000
